# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import math
from collections import defaultdict
from tqdm import tqdm
import json

# Load Data

In [15]:
import pandas as pd

df = pd.read_csv('../../dataset/weighted_score_above_08.csv')

df.head()

/var/folders/sr/kk6cvpy92zvfzcj5gr5l6c6h0000gn/T/ipykernel_81116/410107815.py:3: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../../dataset/weighted_score_above_08.csv')


,recommendationid,appid,game,author_steamid,author_num_games_owned,author_num_reviews,author_playtime_forever,author_playtime_last_two_weeks,author_playtime_at_review,author_last_played,...,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,hidden_in_steam_china,steam_china_location
0,147449116,10,Counter-Strike,76561199183984450,51,12,2548,0,2480,1696305457,...,1,99,2,0.889438,0,1,0,0,1,NaN
1,147374264,10,Counter-Strike,76561198099573060,226,13,2369,0,2361,1696096555,...,1,122,8,0.914834,0,1,0,0,1,NaN
2,147357703,10,Counter-Strike,76561199080026894,118,23,13501,212,12957,1697630734,...,1,599,20,0.968375,3,1,0,0,1,NaN
3,147345102,10,Counter-Strike,76561198068970227,28,1,10668,640,9906,1698261011,...,1,59,12,0.826206,0,0,0,0,1,NaN
4,147284743,10,Counter-Strike,76561199137893460,19,5,543,10,526,1697810991,...,1,128,24,0.853612,0,1,0,0,1,NaN


In [16]:
with open('../combined_df.json', 'r') as f:
    sample_data = [json.loads(line) for line in f]

In [ ]:
# Extract valid appids from the sample file
valid_appids = {entry['appid'] for entry in sample_data}

# Filter df based on appids in the sample
df = df[df['appid'].isin(valid_appids)]

# Continue processing
df_games = df[['appid', 'game']]
df_games = df_games.drop_duplicates(subset='appid')

appid2game = df_games.set_index('appid')['game'].to_dict()
game2appid = df_games.set_index('game')['appid'].to_dict()

del df_games

df = df[['appid', 'author_steamid', 'voted_up']]
df.rename(columns={'appid': 'item_id', 'author_steamid': 'user_id', 'voted_up': 'rating'}, inplace=True)

In [18]:
appid_to_genres = {entry['appid']: entry['genre'] for entry in sample_data}

In [19]:
df.reset_index(drop=True, inplace=True)

In [22]:
N_DISPLAY = 30
has_no_genre_count = 0
for i, appid in enumerate(df['item_id'].unique()):
    genres = appid_to_genres.get(appid, [])
    if not genres:
        has_no_genre_count += 1
        continue
print(f"Number of games without genres: {has_no_genre_count}")

Number of games without genres: 0


In [23]:
df

,item_id,user_id,voted_up
0,10,76561199183984450,1
1,10,76561198099573060,1
2,10,76561199080026894,1
3,10,76561198068970227,1
4,10,76561199137893460,1
...,...,...,...
498089,997010,76561198055740634,1
498090,997010,76561198107142741,1
498091,997060,76561198091884692,1
498092,997180,76561198136664035,1
